In [1]:
# Open the connection to be the database

import psycopg2
import csv
from psycopg2.extras import RealDictCursor

%load_ext sql
%env DATABASE_URL=postgresql://localhost/vp_database1

env: DATABASE_URL=postgresql://localhost/vp_database1


In [2]:
# Create tables that needed

with psycopg2.connect(host='localhost', dbname='vp_database1') as conn:
    with conn.cursor(cursor_factory=RealDictCursor) as cursor:

        cursor.execute(
            """
            DROP TABLE IF EXISTS name_popularity;
            CREATE TABLE name_popularity (
                 id SERIAL PRIMARY KEY,     
                 demographic_group_id INT,
                 name_recipient_group TEXT, -- "baby", "adult" <-- also "dog" if ever add dogs.
                 name_id INT, 
                 gender TEXT, -- unspecified for adults/dogs, male or female for baby names.
                 year INT,
                 count INT
             )
            """
        )
        
        cursor.execute(
            """
            DROP TABLE IF EXISTS name;
            CREATE TABLE name (
                id SERIAL PRIMARY KEY,
                name TEXT
            )
            """
        )
        # use same pattern for synonyms for demographic_group as discussed here for country_names:
        # http://howisonlab.github.io/datawrangling/faq.html#handle-names-with-different-forms
        # always do look ups in the demographic_group_name table.
        cursor.execute(
            """
            DROP TABLE IF EXISTS demographic_group;
            CREATE TABLE demographic_group (
                id INTEGER PRIMARY KEY,
                canonical_name VARCHAR(30) -- "hispanic", "white", "black", "asian"
            )
            """
        )
        
        # manual inserts demographic_group
        cursor.execute(    
            
            """
            INSERT INTO demographic_group (
                id, canonical_name)
            VALUES
                (0, 'Not Human'),
                (1, 'Hispanic'),
                (2, 'White'),
                (3, 'Asian and Pacific Islander'),
                (4, 'Black');
            """
            
        )
        cursor.execute(
            """
            DROP TABLE IF EXISTS demographic_group_name;
            CREATE TABLE demographic_group_name (
                id INTEGER PRIMARY KEY,
                alt_name VARCHAR(30), -- "hispanic", "white", "White Non-Hispanic", "black", "asian", 
                demographic_group_id INTEGER
            )
            """
        )
        
        # manual inserts demographic_group_name (including id)
        cursor.execute(    
            
            """
            INSERT INTO demographic_group_name (
                id, alt_name, demographic_group_id)
            VALUES
                (0, 'unspecified', 0),
                (1, 'HISPANIC', 1),
                (2, 'WHITE NON HISPANIC', 2),
                (3, 'ASIAN AND PACIFIC ISLANDER', 3),
                (4, 'BLACK NON HISPANIC', 4),
                (5, 'ASIAN AND PACI', 3),
                (6, 'BLACK NON HISP', 4),
                (7, 'WHITE NON HISP', 2),
                (8, 'pcthispanic', 1),
                (9, 'pctwhite', 2),
                (10, 'pctaian', 3),
                (11, 'pctblack', 4);
            """
            
        )
        
        # http://howisonlab.github.io/datawrangling/class_modules/12_csv_to_sql_via_python/#our-csv-contains-names-but-we-want-ids
        

In [3]:
# Import Baby Names into name table


with psycopg2.connect(host='localhost', dbname='vp_database1') as conn:
    with conn.cursor(cursor_factory=RealDictCursor) as cursor:
        cursor.execute("TRUNCATE name;")

        with open('BabyNames.csv') as csvfile:
          # tell python about the specific csv format
          myCSVReader = csv.DictReader(csvfile, delimiter=",", quotechar='"')

          # move row by row through the file as though it was a list.
          for row in myCSVReader:
                
                row['Childs_First_Name'] = row['Childs_First_Name'].lower()
                cursor.execute("SELECT id FROM name WHERE name = %(Childs_First_Name)s", 
                               row)

                # How many results?
                if (cursor.rowcount == 1):
                    # Only one result expected, use cursor.fetchone()
                    name_id = cursor.fetchone()['id']
                else:
                    cursor.execute("""
                                    INSERT INTO name(name)
                                    VALUES (
                                         %(Childs_First_Name)s
                                    ) 
                                    RETURNING id
                                    """,
                                    row)  
                    # store the id for the new venue
                    name_id = cursor.fetchone()['id']
print("Done")

Done


In [4]:
# Import Adult Names into name table


with psycopg2.connect(host='localhost', dbname='vp_database1') as conn:
    with conn.cursor(cursor_factory=RealDictCursor) as cursor:

        with open('FirstNames.csv') as csvfile:
          # tell python about the specific csv format
          myCSVReader = csv.DictReader(csvfile, delimiter=",", quotechar='"')

          # move row by row through the file as though it was a list.
          for row in myCSVReader:
                
                row['firstname'] = row['firstname'].lower()
                cursor.execute("SELECT id FROM name WHERE name = %(firstname)s", 
                               row)

                # How many results?
                if (cursor.rowcount == 1):
                    # Only one result expected, use cursor.fetchone()
                    name_id = cursor.fetchone()['id']
                else:
                    cursor.execute("""
                                    INSERT INTO name(name)
                                    VALUES (
                                         %(firstname)s
                                    ) 
                                    RETURNING id
                                    """,
                                    row)  
                    # store the id for the new venue
                    name_id = cursor.fetchone()['id']
print("Done")

Done


In [5]:
# Import Dog Names into name table


with psycopg2.connect(host='localhost', dbname='vp_database1') as conn:
    with conn.cursor(cursor_factory=RealDictCursor) as cursor:

        with open('DogNames.csv') as csvfile:
          # tell python about the specific csv format
          myCSVReader = csv.DictReader(csvfile, delimiter=",", quotechar='"')

          # move row by row through the file as though it was a list.
          for row in myCSVReader:
                
                row['Row_Labels'] = row['Row_Labels'].lower()
                cursor.execute("SELECT id FROM name WHERE name = %(Row_Labels)s", 
                               row)

                # How many results?
                if (cursor.rowcount == 1):
                    # Only one result expected, use cursor.fetchone()
                    name_id = cursor.fetchone()['id']
                else:
                    cursor.execute("""
                                    INSERT INTO name(name)
                                    VALUES (
                                         %(Row_Labels)s
                                    ) 
                                    RETURNING id
                                    """,
                                    row)  
                    # store the id for the new venue
                    name_id = cursor.fetchone()['id']
print("Done")

Done


In [6]:
# Adult name

sql_select_name = "SELECT id from name WHERE name = %(firstname)s"


with psycopg2.connect(host='localhost', dbname='vp_database1') as conn:
    with conn.cursor(cursor_factory=RealDictCursor) as cursor:
        # empty some tables so that we can run this from a known state. 
        
        with open('FirstNames.csv') as csvfile:
            # tell python about the specific csv format
            myCSVReader = csv.DictReader(csvfile, delimiter=",", quotechar='"')

            
            for row in myCSVReader:
                # calculate the count for the group.
                # first, obs, pcthispanic, pctwhite
                # need the count of names in each group, so multipy the pct by the observations.
               
                #param_dict['name'] = row['firstname'] # eventually look up an id.
                
                row['firstname'] = row['firstname'].lower()
                cursor.execute(sql_select_name, row)
                name_id = cursor.fetchone()['id']
                
                total_adult_count = int(row['obs']) # true for all columns
                
                for column in ["pcthispanic", "pctwhite","pctblack","pctaian"]:
                    #param_dict['group'] = column[3:] # pctwhite --> white
                    
                    lookup_sql = """
                        SELECT demographic_group_id
                        FROM demographic_group_name
                        WHERE alt_name = %(column)s
                        """
                    param_dict = {'column': column}
                    cursor.execute(lookup_sql, param_dict)
                    
                    demographic_group_id = cursor.fetchone()['demographic_group_id']
                     
                    pct_group = float(row[column])
                    # if column[:] == "pcthispanic":
                    #     demographic_group_id = "01"
                    # elif column[:] == "pctwhite":
                    #     demographic_group_id = "02"
                    # elif column[:] == "pctaian":
                    #     demographic_group_id = "03"
                    # else:
                    #     demographic_group_id = "04"
                    
                    
                    param_dict = {'name_id': name_id,
                                 'gender': "unspecified",
                                 'demographic_group_id': demographic_group_id,
                                 'name_recipient_group': "adult",
                                 'year': "2018"}
                   
                
                    param_dict['count'] = round(pct_group * total_adult_count)   
                    
                    
                    # use a SELECT query to see if this row is already inserted.
                    cursor.execute("""
                        SELECT *
                        FROM name_popularity
                        WHERE demographic_group_id = %(demographic_group_id)s
                          AND name_recipient_group = %(name_recipient_group)s
                          AND name_id  = %(name_id)s
                          AND gender = %(gender)s
                          AND year = %(year)s
                    """,
                    param_dict)
                    
                    if (cursor.rowcount == 0):
                        cursor.execute("""
                          INSERT INTO name_popularity(
                                    demographic_group_id,
                                    name_recipient_group,
                                    name_id,
                                    gender,
                                    year,
                                    count
                                ) VALUES (
                                    %(demographic_group_id)s,
                                    %(name_recipient_group)s,
                                    %(name_id)s,
                                    %(gender)s,
                                    %(year)s,
                                    %(count)s
                                )
                                """,
                                   param_dict
                        )
                    else:
                        cursor.execute("""
                            UPDATE name_popularity
                            SET count = count + %(count)s
                            WHERE demographic_group_id = %(demographic_group_id)s
                              AND name_recipient_group = %(name_recipient_group)s
                              AND name_id  = %(name_id)s
                              AND gender = %(gender)s
                              AND year = %(year)s
                              """,
                            param_dict
                        )
                    
print("Done loading")

Done loading


In [7]:
# Baby Names

#sql_select_demographic_group_name = "SELECT id from demographic_group_name WHERE alt_name = %(Ethnicity)s"
sql_select_name = "SELECT id from name WHERE name = %(Childs_First_Name)s"
sql_select_demographic_group = "SELECT demographic_group_id from demographic_group_name WHERE alt_name = %(Ethnicity)s"


with psycopg2.connect(host='localhost', dbname='vp_database1') as conn:
    with conn.cursor(cursor_factory=RealDictCursor) as cursor:
        #cursor.execute("TRUNCATE name_popularity;")# empty some tables so that we can run this from a known state. 
        
        with open('BabyNames.csv') as csvfile:
            # tell python about the specific csv format
            myCSVReader = csv.DictReader(csvfile, delimiter=",", quotechar='"')

            #param_dict = {'age_group': "baby",
                         #'demographic_group_name_id': demographic_group_name_id} # true for whole file.
            
            for row in myCSVReader:
                
                cursor.execute(sql_select_demographic_group, row)
                demographic_group_id = cursor.fetchone()['demographic_group_id']
                
                row['Childs_First_Name'] = row['Childs_First_Name'].lower()
                cursor.execute(sql_select_name, row)
                name_id = cursor.fetchone()['id']
                
                
                param_dict = {'name_recipient_group': "baby",
                            'demographic_group_id': demographic_group_id,
                             'name_id' : name_id,
                             'count' : row['Count'],
                             'gender' : row['Gender'],
                             'year': row['Year_of_Birth']} # true for whole file.
                
                #param_dict['name'] = row['Childs_First_Name'] # eventually look up an id.
                #param_dict['count'] = row['Count']
                #param_dict['gender'] = row['Gender']
                # use a SELECT query to see if this row is already inserted.
                cursor.execute("""
                        SELECT *
                        FROM name_popularity
                        WHERE demographic_group_id = %(demographic_group_id)s
                          AND name_recipient_group = %(name_recipient_group)s
                          AND name_id  = %(name_id)s
                          AND gender = %(gender)s
                          AND year = %(year)s
                    """,
                    param_dict)
                    
                if (cursor.rowcount == 0):
                    cursor.execute("""
                          INSERT INTO name_popularity(
                                    demographic_group_id,
                                    name_recipient_group,
                                    name_id,
                                    gender,
                                    year,
                                    count
                                ) VALUES (
                                    %(demographic_group_id)s,
                                    %(name_recipient_group)s,
                                    %(name_id)s,
                                    %(gender)s,
                                    %(year)s,
                                    %(count)s
                                )
                                """,
                                   param_dict
                        )
                else:
                    cursor.execute("""
                            UPDATE name_popularity
                            SET count = count + %(count)s
                            WHERE demographic_group_id = %(demographic_group_id)s
                              AND name_recipient_group = %(name_recipient_group)s
                              AND name_id  = %(name_id)s
                              AND gender = %(gender)s
                              AND year = %(year)s
                              """,
                            param_dict
                        )
                    
                    
                
print("Done loading")

Done loading


In [8]:
# Dog Names

sql_select_name = "SELECT id from name WHERE name = %(Row_Labels)s"


with psycopg2.connect(host='localhost', dbname='vp_database1') as conn:
    with conn.cursor(cursor_factory=RealDictCursor) as cursor:
        #cursor.execute("TRUNCATE name_popularity;")# empty some tables so that we can run this from a known state. 
        
        with open('DogNames.csv') as csvfile:
            # tell python about the specific csv format
            myCSVReader = csv.DictReader(csvfile, delimiter=",", quotechar='"')

            #param_dict = {'age_group': "baby",
                         #'demographic_group_name_id': demographic_group_name_id} # true for whole file.
            
            for row in myCSVReader:
                
                row['Row_Labels'] = row['Row_Labels'].lower()
                cursor.execute(sql_select_name, row)
                name_id = cursor.fetchone()['id']
                
                
                param_dict = {'name_recipient_group': "dog",
                             'demographic_group_id': 0,  # Not Human
                             'name_id' : name_id,
                             'count' : row['Count_AnimalName'],
                             'gender' : "unspecified",
                             'year': "2018"} # true for whole file.
        
                cursor.execute("""
                        SELECT *
                        FROM name_popularity
                        WHERE demographic_group_id = %(demographic_group_id)s
                          AND name_recipient_group = %(name_recipient_group)s
                          AND name_id  = %(name_id)s
                          AND gender = %(gender)s
                          AND year = %(year)s
                    """,
                    param_dict)
                    
                if (cursor.rowcount == 0):
                    cursor.execute("""
                          INSERT INTO name_popularity(
                                    demographic_group_id,
                                    name_recipient_group,
                                    name_id,
                                    gender,
                                    year,
                                    count
                                ) VALUES (
                                    %(demographic_group_id)s,
                                    %(name_recipient_group)s,
                                    %(name_id)s,
                                    %(gender)s,
                                    %(year)s,
                                    %(count)s
                                )
                                """,
                                   param_dict
                        )
                else:
                    cursor.execute("""
                            UPDATE name_popularity
                            SET count = count + %(count)s
                            WHERE demographic_group_id = %(demographic_group_id)s
                              AND name_recipient_group = %(name_recipient_group)s
                              AND name_id  = %(name_id)s
                              AND gender = %(gender)s
                              AND year = %(year)s
                              """,
                            param_dict
                        )
                    
                
print("Done loading")

Done loading


In [9]:
%%sql

SELECT name_popularity.id, count, name_recipient_group, year
FROM name_popularity 
JOIN demographic_group
  ON demographic_group_id = name_popularity.demographic_group_id

JOIN name
  ON name.id = name_popularity.name_id
GROUP BY name_popularity.id
LIMIT 30

30 rows affected.


id,count,name_recipient_group,year
1,10500,adult,2018
2,333999,adult,2018
3,11901,adult,2018
4,201,adult,2018
5,0,adult,2018
6,4200,adult,2018
7,200,adult,2018
8,0,adult,2018
9,0,adult,2018
10,5500,adult,2018


In [10]:
%%sql

SELECT name_popularity.id, count, name_recipient_group, year
FROM name_popularity 
JOIN demographic_group
  ON demographic_group_id = name_popularity.demographic_group_id
JOIN demographic_group_name
  ON demographic_group_name.demographic_group_id = demographic_group.id
JOIN name
  ON name.id = name_popularity.name_id
WHERE name_popularity.year = '2018'and name_popularity.name_recipient_group = 'adult' and name_popularity.demographic_group_id = '01'
GROUP BY name_popularity.id
ORDER BY name_popularity.count desc
LIMIT 20

 * postgresql://localhost/vp_database1
20 rows affected.


id,count,name_recipient_group,year
17001,1761384,adult,2018
7865,776302,adult,2018
10481,661997,adult,2018
7941,375399,adult,2018
2353,239100,adult,2018
10029,235800,adult,2018
7857,176001,adult,2018
7549,174200,adult,2018
5381,165600,adult,2018
11189,162700,adult,2018


In [11]:
%%sql

SELECT *
FROM name_popularity 
JOIN name
  ON name.id = name_popularity.name_id
JOIN demographic_group
  ON demographic_group_id = name_popularity.demographic_group_id
JOIN demographic_group_name
  ON demographic_group_name.demographic_group_id = demographic_group.id
ORDER BY name.name desc

LIMIT 20

 * postgresql://localhost/vp_database1
20 rows affected.


id,demographic_group_id,name_recipient_group,name_id,gender,year,count,id_1,name,id_2,canonical_name,id_3,alt_name,demographic_group_id_1
49302,0,dog,18902,unspecified,2018,2,18902,zz,0,Not Human,0,unspecified,0
49302,0,dog,18902,unspecified,2018,2,18902,zz,1,Hispanic,8,pcthispanic,1
49302,0,dog,18902,unspecified,2018,2,18902,zz,4,Black,11,pctblack,4
49302,0,dog,18902,unspecified,2018,2,18902,zz,1,Hispanic,1,HISPANIC,1
49302,0,dog,18902,unspecified,2018,2,18902,zz,4,Black,4,BLACK NON HISPANIC,4
49302,0,dog,18902,unspecified,2018,2,18902,zz,2,White,2,WHITE NON HISPANIC,2
49302,0,dog,18902,unspecified,2018,2,18902,zz,2,White,9,pctwhite,2
49302,0,dog,18902,unspecified,2018,2,18902,zz,3,Asian and Pacific Islander,10,pctaian,3
49302,0,dog,18902,unspecified,2018,2,18902,zz,4,Black,6,BLACK NON HISP,4
49302,0,dog,18902,unspecified,2018,2,18902,zz,3,Asian and Pacific Islander,3,ASIAN AND PACIFIC ISLANDER,3


In [ ]:
with psycopg2.connect(host='localhost', dbname='vp_database1') as conn:
    with conn.cursor(cursor_factory=RealDictCursor) as cur:

        cur.execute(
            """
            SELECT *
                FROM (
                    SELECT  name_popularity.year,
                            name_popularity.name_recipient_group,
                            demographic_group.canonical_name AS demo_group,
                            name.name AS name,
                            name_popularity.gender,
                            count,
                            DENSE_RANK () OVER (                                  
                                PARTITION BY year, 
                                             name_recipient_group, 
                                             demographic_group.canonical_name,
                                             name_popularity.gender
                                ORDER BY count desc
                            ) AS name_rank  

                    FROM name_popularity 
                    JOIN name
                        ON name.id = name_popularity.name_id
                    JOIN demographic_group
                        ON demographic_group.id = name_popularity.demographic_group_id

                ) AS name_rankings
                WHERE name_rank <= 30 and (year = 2019 or year = 2018)
            """)
        result = cur.fetchall()
        csv_olumn_names = list(result[0].keys())

        with open('Result.csv', 'w', newline='') as csvfile:
                    # Declare csv fields in the order we want them
            #column_names = [i[0] for i in cursor.description]
            myCsvWriter = csv.DictWriter(csvfile,
                                                 delimiter = ',',
                                                fieldnames = csv_olumn_names)

            myCsvWriter.writeheader()

            for row in cur:          
                myCsvWriter.writerow(row)

print("Done writing csv")

Done writing csv


In [25]:
with psycopg2.connect(host='localhost', dbname='vp_database1') as conn:
    with conn.cursor(cursor_factory=RealDictCursor) as cur:

        cur.execute(
            """
             SELECT *
                FROM (
                    SELECT  name_popularity.year,
                            name_popularity.name_recipient_group,
                            demographic_group.canonical_name AS demographic_group,
                            name.name AS name,
                            name_popularity.gender,
                            count,
                            DENSE_RANK () OVER (                                  
                                PARTITION BY year, 
                                             name_recipient_group, 
                                             demographic_group.canonical_name,
                                             name_popularity.gender
                                ORDER BY count desc
                            ) AS name_rank  

                    FROM name_popularity 
                    JOIN name
                        ON name.id = name_popularity.name_id
                    JOIN demographic_group
                        ON demographic_group.id = name_popularity.demographic_group_id

                ) AS name_rankings
                WHERE name_rank <= 30 and (year = 2019 or year = 2018)

            """)
        #result = cur.fetchall()
        #csv_olumn_names = list(result[0].keys())

        with open('result.csv', 'w', newline='') as csvfile:
                    # Declare csv fields in the order we want them
            column_names = ["year", "name_recipient_group", "demographic_group", "name", "gender", "count", "name_rank"]
            myCsvWriter = csv.DictWriter(csvfile,
                                                 delimiter = ',',
                                                fieldnames = column_names)

            myCsvWriter.writeheader()

            for row in cur:          
                myCsvWriter.writerow(row)

print("Done writing csv")

Done writing csv


Eventually we want the ranking of a name babies vs adults for each demo_group.

name_recipient_group, demographic_group.canonical_name, name_rank
adult,AARON,white,8
baby,AARON,white,3

WHERE name_rank <= 10


ORDER BY name_recipient_group, demographic_group.canonical_name, count


In [12]:
%%sql

SELECT  name_popularity.year,
        name_popularity.name_recipient_group,
        demographic_group.canonical_name AS demo_group,
        name.name AS name,
        count
        
FROM name_popularity 
JOIN name
    ON name.id = name_popularity.name_id
JOIN demographic_group
    ON demographic_group.id = name_popularity.demographic_group_id

ORDER BY name_popularity.year,
    name_popularity.name_recipient_group,
    demographic_group.canonical_name,
    name.name, 
    count DESC
    
LIMIT 20

 * postgresql://localhost/vp_database1
20 rows affected.


year,name_recipient_group,demo_group,name,count
2011,baby,Asian and Pacific Islander,aarav,60
2011,baby,Asian and Pacific Islander,aaron,204
2011,baby,Asian and Pacific Islander,abdul,80
2011,baby,Asian and Pacific Islander,abdullah,120
2011,baby,Asian and Pacific Islander,abigail,96
2011,baby,Asian and Pacific Islander,ada,52
2011,baby,Asian and Pacific Islander,adam,112
2011,baby,Asian and Pacific Islander,aditya,56
2011,baby,Asian and Pacific Islander,adrian,76
2011,baby,Asian and Pacific Islander,ahmed,84


In [ ]:
%%sql

-- gender only for babies, need to collapse across gender for babies.

SELECT *
FROM (
    SELECT  name_popularity.year,
            name_popularity.name_recipient_group,
            demographic_group.canonical_name AS demographic_group,
            name.name AS name,
            name_popularity.gender,
            count,
            DENSE_RANK () OVER (                                  
                PARTITION BY year, 
                             name_recipient_group, 
                             demographic_group.canonical_name,
                             name_popularity.gender
                ORDER BY count desc
            ) AS name_rank  

    FROM name_popularity 
    JOIN name
        ON name.id = name_popularity.name_id
    JOIN demographic_group
        ON demographic_group.id = name_popularity.demographic_group_id
    
) AS name_rankings
WHERE name_rank <= 30 and (year = 2019 or year = 2018)
LIMIT 10

In [14]:
%%sql

SELECT *
FROM name_popularity 
JOIN name
  ON name.id = name_popularity.name_id
JOIN demographic_group
  ON demographic_group.id = name_popularity.demographic_group_id

    
LIMIT 20

 * postgresql://localhost/vp_database1
20 rows affected.


id,demographic_group_id,name_recipient_group,name_id,gender,year,count,id_1,name,id_2,canonical_name
1,1,adult,464,unspecified,2018,10500,464,aaron,1,Hispanic
2,2,adult,464,unspecified,2018,333999,464,aaron,2,White
3,4,adult,464,unspecified,2018,11901,464,aaron,4,Black
4,3,adult,464,unspecified,2018,201,464,aaron,3,Asian and Pacific Islander
5,1,adult,1903,unspecified,2018,0,1903,abbas,1,Hispanic
6,2,adult,1903,unspecified,2018,4200,1903,abbas,2,White
7,4,adult,1903,unspecified,2018,200,1903,abbas,4,Black
8,3,adult,1903,unspecified,2018,0,1903,abbas,3,Asian and Pacific Islander
9,1,adult,1904,unspecified,2018,0,1904,abbey,1,Hispanic
10,2,adult,1904,unspecified,2018,5500,1904,abbey,2,White
